# 02. 예약률 심층 분석 (Booking Rate Analysis)

요일별/시간대별/시즌별 예약 패턴을 분석합니다.

In [ ]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from models.database import session_scope
from models.schema import Station, DailyStat

In [ ]:
with session_scope() as session:
    stations = pd.read_sql(session.query(Station).statement, session.bind)
    daily_stats = pd.read_sql(session.query(DailyStat).statement, session.bind)

daily_stats['date'] = pd.to_datetime(daily_stats['date'])
total_stats = daily_stats[daily_stats['room_type'].isna()].copy()
total_stats['weekday'] = total_stats['date'].dt.day_name()
total_stats['month'] = total_stats['date'].dt.month
total_stats['week_num'] = total_stats['date'].dt.isocalendar().week

print(f'분석 기간: {total_stats["date"].min().date()} ~ {total_stats["date"].max().date()}')

## 1. 요일별 예약률

In [ ]:
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
weekday_stats = (
    total_stats.groupby('weekday')['booking_rate']
    .mean()
    .reindex(weekday_order)
    .reset_index()
)
weekday_stats.columns = ['weekday', 'avg_booking_rate']

fig = px.bar(weekday_stats, x='weekday', y='avg_booking_rate',
             title='요일별 평균 예약률',
             labels={'weekday': '요일', 'avg_booking_rate': '평균 예약률'})
fig.update_yaxes(tickformat='.0%')
fig.show()

## 2. 월별 예약률 추이

In [ ]:
monthly_stats = (
    total_stats.groupby('month')['booking_rate']
    .mean()
    .reset_index()
)
monthly_stats.columns = ['month', 'avg_booking_rate']

fig = px.line(monthly_stats, x='month', y='avg_booking_rate',
              title='월별 평균 예약률 추이',
              labels={'month': '월', 'avg_booking_rate': '평균 예약률'},
              markers=True)
fig.update_yaxes(tickformat='.0%')
fig.show()

## 3. 역별 예약률 히트맵

In [ ]:
# 상위 15개 역 × 날짜 히트맵
merged = total_stats.merge(stations[['id', 'name']], left_on='station_id', right_on='id')

top_stations = (
    merged.groupby('name')['booking_rate'].mean()
    .sort_values(ascending=False)
    .head(15)
    .index.tolist()
)

pivot = (
    merged[merged['name'].isin(top_stations)]
    .pivot_table(index='name', columns='date', values='booking_rate', aggfunc='mean')
)

fig = px.imshow(pivot,
                title='역별 일별 예약률 히트맵 (상위 15개 역)',
                labels={'x': '날짜', 'y': '역명', 'color': '예약률'},
                color_continuous_scale='RdYlGn')
fig.show()

## 4. 숙소 유형별 예약률 비교

In [ ]:
room_type_stats = (
    daily_stats[daily_stats['room_type'].notna()]
    .groupby(['date', 'room_type'])['booking_rate']
    .mean()
    .reset_index()
)

fig = px.line(room_type_stats, x='date', y='booking_rate', color='room_type',
              title='숙소 유형별 예약률 추이',
              labels={'date': '날짜', 'booking_rate': '예약률', 'room_type': '숙소 유형'})
fig.update_yaxes(tickformat='.0%')
fig.show()

## 5. 예약률 상관관계 분석

In [ ]:
# 예약률 vs 평균 가격 산점도
fig = px.scatter(total_stats, x='avg_daily_price', y='booking_rate',
                 title='평균 가격 vs 예약률',
                 labels={'avg_daily_price': '평균 일일 가격 (KRW)', 'booking_rate': '예약률'},
                 opacity=0.5,
                 trendline='ols')
fig.update_yaxes(tickformat='.0%')
fig.show()

corr = total_stats[['booking_rate', 'avg_daily_price', 'total_listings']].corr()
print('\n상관관계 행렬:')
print(corr)